***
# Projet n°7 - Implémentez un modèle de scoring
***
Notebook Jupyter réalisé par Dominique LEVRAY en Juillet/Août 2024

**3ème Notebook - Test_API**

Ce notebook intègre :
- Les tests de l'API de prédiction du modèle de scoring
***


In [1]:
# Importation des modules dont on aura besoin
import os
import time
import re
import numpy as np
import pandas as pd
import matplotlib as pl
import matplotlib.pyplot as plt
import pprint

# Change le format d'affichage des floats pour être plus lisible
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Force l'affichage de toutes les colonnes et de toutes les lignes dans les fonctions head/tail/dtypes/etc... 
pd.set_option('display.max_columns',  None)     # Afficher toutes les colonnes
pd.set_option('display.max_rows',     None)     # Afficher toutes les lignes
pd.set_option('display.max_colwidth', 40)       # Limite sur la largeur des colonnes

# Défini les tailles par défaut des polices de caractères pour tous les diagrammes matplotlib
plt.rcParams.update({'font.size': 10}) 
plt.rcParams.update({'axes.titlesize': 12})
plt.rcParams.update({'axes.labelsize': 10})
plt.rcParams.update({'xtick.labelsize': 8})
plt.rcParams.update({'ytick.labelsize': 8})
plt.rcParams.update({'legend.fontsize': 8})

# Défini les grilles par défaut pour tous les diagrammes matplotlib
plt.rcParams.update({"axes.grid" : True, 'grid.color': 'green', 'grid.linestyle':'--', 'grid.linewidth':0.5})

In [2]:
WANTED_PATH="D:\\Dominique\\GIT\\OC_DS_P7"
os.chdir(WANTED_PATH)
print("Répertoire courant : "+os.getcwd())

API_FOLDER = "levray_dominique_3_dossier_code_072024"
ZIP_TEST_DATA_FILENAME = "test_data.zip"                 # fichier contenant les données de test

data_df  = pd.read_csv(API_FOLDER+"/"+ZIP_TEST_DATA_FILENAME,  sep=',', encoding='utf-8',compression='zip')
print(f"\tTaille de data_df={data_df.shape}")


Répertoire courant : D:\Dominique\GIT\OC_DS_P7
	Taille de data_df=(30752, 166)


In [3]:
data_df[data_df['TARGET']==0].head(10)

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,NAME_CONTRACT_TYPE,AMT_CREDIT,CNT_CHILDREN,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,DAYS_BIRTH_x,FLAG_EMAIL,AMT_ANNUITY,REG_CITY_NOT_WORK_CITY,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,FLOORSMIN_MEDI,FLAG_MOBIL,YEARS_BEGINEXPLUATATION_AVG,BASEMENTAREA_AVG,DAYS_EMPLOYED,NONLIVINGAREA_AVG,FLAG_EMP_PHONE,DAYS_ID_PUBLISH,FLAG_CONT_MOBILE,DAYS_REGISTRATION,APARTMENTS_AVG,NONLIVINGAPARTMENTS_AVG,ENTRANCES_AVG,YEARS_BUILD_MEDI,REG_CITY_NOT_LIVE_CITY,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,LIVE_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,LIVE_REGION_NOT_WORK_REGION,FLAG_WORK_PHONE,EXT_SOURCE_2_x,EXT_SOURCE_1_x,FLOORSMAX_AVG,FLAG_PHONE,COMMONAREA_AVG,LANDAREA_MODE,CNT_FAM_MEMBERS,YEARS_BUILD_AVG,ENTRANCES_MODE,ELEVATORS_AVG,BASEMENTAREA_MODE,AMT_REQ_CREDIT_BUREAU_WEEK,YEARS_BEGINEXPLUATATION_MODE,LANDAREA_MEDI,ELEVATORS_MODE,YEARS_BUILD_MODE,REG_REGION_NOT_WORK_REGION,FLOORSMIN_AVG,NONLIVINGAREA_MODE,EXT_SOURCE_3_x,DEF_30_CNT_SOCIAL_CIRCLE,LIVINGAPARTMENTS_AVG,LANDAREA_AVG,FLOORSMAX_MODE,NONLIVINGAPARTMENTS_MEDI,COMMONAREA_MEDI,APARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAPARTMENTS_MEDI,FLOORSMIN_MODE,FLAG_DOCUMENT_6,ENTRANCES_MEDI,EMERGENCYSTATE_MODE,ELEVATORS_MEDI,COMMONAREA_MODE,BASEMENTAREA_MEDI,FLAG_DOCUMENT_7,DAYS_LAST_PHONE_CHANGE,YEARS_BEGINEXPLUATATION_MEDI,NONLIVINGAPARTMENTS_MODE,APARTMENTS_MEDI,OBS_30_CNT_SOCIAL_CIRCLE,LIVINGAREA_MEDI,FLOORSMAX_MEDI,FLAG_DOCUMENT_3,_SEC_CNT_DRAWINGS_ATM_CURRENT,_SEC_AMT_CREDIT_SUM_LIMIT,LIVINGAPARTMENTS_MODE,FLAG_DOCUMENT_16,FLAG_DOCUMENT_11,AMT_REQ_CREDIT_BUREAU_MON,_SEC_AMT_CREDIT_SUM_DEBT,NONLIVINGAREA_MEDI,LIVINGAREA_MODE,FLAG_DOCUMENT_4,_SEC_INCIDENTS_CREDITS_CLOS,_SEC_AMT_CREDIT_SUM_OVERDUE,_SEC_AMT_CREDIT_LIMIT_ACTUAL,TOTALAREA_MODE,NAME_HOUSING_TYPE_Withparents,FLAG_DOCUMENT_15,FLAG_DOCUMENT_13,FLAG_DOCUMENT_10,AMT_REQ_CREDIT_BUREAU_DAY,_SEC_AMT_RECIVABLE,_SEC_AMT_DRAWINGS_CURRENT,NAME_TYPE_SUITE_Other_A,FLAG_DOCUMENT_9,FLAG_DOCUMENT_21,FLAG_DOCUMENT_19,FLAG_DOCUMENT_17,FLAG_DOCUMENT_14,FLAG_DOCUMENT_12,DEF_60_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,NAME_EDUCATION_TYPE_Academicdegree,AMT_REQ_CREDIT_BUREAU_QRT,_SEC_DAYS_CREDIT_UPDATE,_SEC_DAYS_CREDIT,_SEC_AMT_TOTAL_RECEIVABLE,FLAG_DOCUMENT_8,FLAG_DOCUMENT_5,_SEC_CREDIT_DAY_OVERDUE,_SEC_CNT_INSTALMENT,_SEC_CNT_CREDIT_PROLONG,_SEC_AMT_PAYMENT_CURRENT,_SEC_AMT_CREDIT_SUM,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Groupofpeople,AMT_REQ_CREDIT_BUREAU_YEAR,_SEC_CREDITS_CLOS,_SEC_CNT_DRAWINGS_POS_CURRENT,_SEC_CNT_DRAWINGS_OTHER_CURRENT,_SEC_AMT_INST_MIN_REGULARITY,_SEC_AMT_DRAWINGS_POS_CURRENT,OCCUPATION_TYPE_Cleaningstaff,OCCUPATION_TYPE_Accountants,NAME_INCOME_TYPE_Businessman,NAME_FAMILY_STATUS_Civilmarriage,FLAG_DOCUMENT_20,FLAG_DOCUMENT_2,FLAG_DOCUMENT_18,_SEC_DAYS_ENDDATE_FACT,_SEC_DAYS_CREDIT_ENDDATE,_SEC_CNT_INSTALMENT_FUTURE,NAME_TYPE_SUITE_Family,NAME_HOUSING_TYPE_Coopapartment,AMT_REQ_CREDIT_BUREAU_HOUR,_SEC_CREDITS_ACTIFS,_SEC_AMT_RECEIVABLE_PRINCIPAL,NAME_TYPE_SUITE_Spousepartner,NAME_INCOME_TYPE_Unemployed,NAME_FAMILY_STATUS_Singlenotmarried,_SEC_AMT_PAYMENT_TOTAL_CURRENT,_SEC_AMT_DRAWINGS_ATM_CURRENT,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Separated,NAME_EDUCATION_TYPE_Incompletehigher,DAYS_EMPLOYED_ANOM,_SEC_INCIDENTS_CREDITS_ACTIFS,_SEC_CNT_DRAWINGS_CURRENT,_SEC_AMT_DRAWINGS_OTHER_CURRENT,OCCUPATION_TYPE_LowskillLaborers,OCCUPATION_TYPE_Corestaff,OCCUPATION_TYPE_Cookingstaff,NAME_TYPE_SUITE_Children,TARGET
0,190501,1,1,1,675000.000,0,0,225000.000,675000.000,12,-18542,0,33750.000,0,0.007,4.000,0.375,1,0.989,0.126,-5624.000,0.000,1,-2070,1,-6011.000,0.186,0.009,0.172,0.846,0,2,2,0,0,0,0,0.675,0.502,0.333,1,0.045,0.026,2.000,0.844,0.172,0.200,0.130,0.000,0.989,0.026,0.201,0.850,0,0.375,0.000,0.800,0.000,0.101,0.026,0.333,0.009,0.045,0.189,0.208,0.102,0.375,0,0.172,0,0.200,0.043,0.126,0,0.000,0.989,0.008,0.187,0.000,0.211,0.333,0,0.000,0.000,0.106,0,0,0.000,0.000,0.000,0.216,0,0.000,0.000,0.000,0.1